In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
import pandas as pd
import numpy as np
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder


test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)


In [ ]:
print(test.columns.values)
#['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin' 'Embarked']
print(train.columns.values)
#['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin' 'Embarked']
print(len(test))

In [ ]:
#remove unnecessary data: "ticket"
for df in [test,train]:
  u_data = ["Cabin","Ticket","Name"]
  for u in u_data:
    df.drop(u,axis=1,inplace=True)

In [ ]:
train.dropna(subset=['Fare','Embarked'],inplace=True)

print( type(test['Fare'].isnull())    )
print(type(test['Fare'].isnull().values))

for s in ['Fare','Embarked']:
  most_freq = test[s].value_counts().idxmax()
  for c in range(len(test[s])):
    if(test[s].isnull()[c]):
      test[s][c] = most_freq

#test.dropna(subset=['Fare','Embarked'],inplace=True)

In [ ]:
train.sort_values(by="Age",inplace=True)
train['Age'] = train['Age'].interpolate(method='linear',limit_direction='forward',axis=0)
#https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e

test.sort_values(by="Age",inplace=True)
test['Age'] = test['Age'].interpolate(method='linear',limit_direction='forward',axis=0)
#https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e

In [ ]:
train.dropna(subset=['Sex'])
#test.dropna(subset=['Sex'])

most_freq = test['Sex'].value_counts().idxmax()


#https://datascienceparichay.com/article/most-frequent-value-in-a-pandas-column/
for c in range(len(test['Sex'])):
  if(test['Sex'].isnull().values[c]):
    test['Sex'][c] = most_freq

In [ ]:
print(test['Sex'].value_counts())

In [ ]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [ ]:
#one hot encode Sex

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_df = pd.DataFrame(encoder.fit_transform(train[['Sex']]).toarray())
encoded_df.rename(columns={0:'Female',1:'Male'},inplace=True)


In [ ]:
train = train.drop('Sex', axis=1)
train = train.join(encoded_df)
#print(train.isnull().sum())

In [ ]:
#one hot encode Sex

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_df = pd.DataFrame(encoder.fit_transform(test[['Sex']]).toarray())
encoded_df.rename(columns={0:'Female',1:'Male'},inplace=True)

test = test.drop('Sex', axis=1)
test = test.join(encoded_df)


In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
encoded_df = pd.DataFrame(encoder.fit_transform(train[['Embarked']]))

train.drop('Embarked',axis=1,inplace=True)
train = train.join(encoded_df)


In [ ]:
encoded_df = pd.DataFrame(encoder.fit_transform(test[['Embarked']]))

most_freq = test['Embarked'].value_counts().idxmax();
for r in range(len(test)):
  if(test['Embarked'].isnull()[r]):
    test['Embarked'][r] = most_freq



test.drop('Embarked',axis=1,inplace=True)
test = test.join(encoded_df)

In [ ]:
#handle null/NaN values

for df in [test,train]:
  print("-----------------------------------------")
  print(df.isnull().sum())
  for c in df.columns.values:
    print(df[[c]].isna().sum())

"""
test
-----------------------------------------
Age    86
Fare    1

train
-----------------------------------------
Age    177
Embarked    2
"""


#Strategy: remove fare, embarked and fix age

In [ ]:
import math
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=math.floor(math.sqrt(len(train))))
#X is data, Y is label
X = train.drop(columns=['Survived','PassengerId'])
Y = train[['Survived']]

model.fit(X,Y)
model.score(X,Y)

preds = model.predict(test.drop(columns=['PassengerId']))

#0.63875

In [ ]:
from sklearn import svm

model = svm.SVC()
X = train.drop(columns=['Survived','PassengerId'])
Y = train[['Survived']]

model.fit(X,Y)
model.score(X,Y)

preds = model.predict(test.drop(columns=['PassengerId']))

#0.66985

In [ ]:
from sklearn.naive_bayes import GaussianNB


model = GaussianNB();
X = train.drop(columns=['Survived','PassengerId'])
Y = train[['Survived']]

model.fit(X,Y)
model.score(X,Y)

preds = model.predict(test.drop(columns=['PassengerId']))

#0.75119

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

model = LogisticRegression()

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X = train.drop(columns=['Survived','PassengerId'])
Y = train[['Survived']]
Y = label_encoder.fit_transform(Y)

model.fit(X,Y)
model.score(X,Y)

preds = model.predict(test.drop(columns=['PassengerId']))

#0.75119

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.DataFrame(columns=['PassengerId','Survived']);
df['PassengerId'] = test['PassengerId']
df['Survived'] = preds

df.to_csv('data2.csv',index=False)
!cp data.csv "drive/My Drive/"